<a href="https://colab.research.google.com/github/Apoorvkhanna2/Railway-Reserve/blob/main/Railway_Reservation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import random
import os
from collections import defaultdict
from datetime import datetime

class RailwaySystem:
    def __init__(self, data_file):
        self.data_file = data_file
        self.bookings = []
        self.train_index = defaultdict(list)
        self.df_trains = None
        self.load_data()
        if self.df_trains is None:
            raise ValueError("Train data not loaded successfully.")

    def load_data(self):
        try:
            self.df_trains = pd.read_csv(self.data_file)
            self.df_trains.columns = self.df_trains.columns.str.strip()
            self.df_trains.dropna(subset=['Train no.', 'Train name', 'Starts', 'Ends'], inplace=True)
            for _, row in self.df_trains.iterrows():
                self.train_index['Starts'].append(row['Starts'].strip().lower())
                self.train_index['Ends'].append(row['Ends'].strip().lower())
                self.train_index['Train_No'].append(str(row['Train no.']).strip())
            print(f"Loaded {len(self.df_trains)} trains from {self.data_file}")
        except FileNotFoundError:
            print(f"Error: File {self.data_file} not found.")
            self.df_trains = None
        except Exception as e:
            print(f"Error loading data: {str(e)}")
            self.df_trains = None

    def save_data(self):
        try:
            self.df_trains.to_csv(self.data_file, index=False)
            print("Data saved successfully.")
        except Exception as e:
            print(f"Error saving data: {str(e)}")

    def display_trains(self, trains=None):
        trains_to_display = trains if trains is not None else self.df_trains
        if trains_to_display.empty:
            print("No trains found.")
            return

        page_size = 10
        total_pages = (len(trains_to_display) + page_size - 1) // page_size
        for page in range(total_pages):
            start = page * page_size
            end = start + page_size
            print(f"\n--- Trains {start+1}-{min(end, len(trains_to_display))} of {len(trains_to_display)} ---")
            for _, train in trains_to_display.iloc[start:end].iterrows():
                print(f"{train['Train no.']} - {train['Train name']} | {train['Starts']} ➝ {train['Ends']}")
            if page < total_pages - 1:
                input("\nPress Enter to view more trains...")

    def search_trains(self, query, by='Starts'):
        query = query.lower().strip()
        if by == 'Starts':
            return self.df_trains[self.df_trains['Starts'].str.lower().str.contains(query)]
        elif by == 'Ends':
            return self.df_trains[self.df_trains['Ends'].str.lower().str.contains(query)]
        elif by == 'Name':
            return self.df_trains[self.df_trains['Train name'].str.lower().str.contains(query)]
        elif by == 'Number':
            return self.df_trains[self.df_trains['Train no.'].astype(str).str.lower() == query]
        return pd.DataFrame()

    def find_train_by_number(self, number):
        result = self.df_trains[self.df_trains['Train no.'].astype(str) == number]
        return result.iloc[0] if not result.empty else None

    def add_train(self, train_no, name, start, end):
        if self.find_train_by_number(train_no) is not None:
            print(f"Train number {train_no} already exists.")
            return False
        new_row = pd.DataFrame([[train_no, name, start, end]], columns=self.df_trains.columns)
        self.df_trains = pd.concat([self.df_trains, new_row], ignore_index=True)
        self.train_index['Starts'].append(start.lower())
        self.train_index['Ends'].append(end.lower())
        self.train_index['Train_No'].append(train_no)
        print(f"Train {train_no} added successfully.")
        return True

    def delete_train(self, train_no):
        if self.find_train_by_number(train_no) is None:
            print(f"Train {train_no} not found.")
            return False
        self.df_trains = self.df_trains[self.df_trains['Train no.'].astype(str) != train_no]
        print(f"Train {train_no} deleted successfully.")
        return True

    def generate_pnr(self):
        while True:
            pnr = ''.join(str(random.randint(0, 9)) for _ in range(10))
            if not any(b['PNR'] == pnr for b in self.bookings):
                return pnr

    def book_train(self):
        train_no = input("Enter Train Number to book: ").strip()
        train = self.find_train_by_number(train_no)
        if train is None:
            print("Train not found.")
            return

        name = input("Passenger Name: ").strip()
        age = input("Passenger Age: ").strip()
        gender = input("Passenger Gender (M/F/O): ").strip().upper()

        class_options = {
            '1': {'name': 'Sleeper', 'fare': 500},
            '2': {'name': '3rd AC', 'fare': 1200},
            '3': {'name': '2nd AC', 'fare': 1800},
            '4': {'name': '1st AC', 'fare': 2500}
        }
        for num, cls in class_options.items():
            print(f"{num}. {cls['name']} (₹{cls['fare']})")

        while True:
            class_choice = input("Choose Class (1-4): ").strip()
            if class_choice in class_options:
                travel_class = class_options[class_choice]['name']
                fare = class_options[class_choice]['fare']
                break

        pnr = self.generate_pnr()
        booking = {
            'PNR': pnr,
            'Booking_Date': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'Train_No': train['Train no.'],
            'Train_Name': train['Train name'],
            'From': train['Starts'],
            'To': train['Ends'],
            'Passenger_Name': name,
            'Age': age,
            'Gender': gender,
            'Class': travel_class,
            'Fare': fare,
            'Status': 'Confirmed'
        }
        self.bookings.append(booking)
        print(f"\n✅ Booking Confirmed! PNR: {pnr}")

    def view_all_bookings(self):
        if not self.bookings:
            print("No bookings found.")
            return
        print("\n📘 All Bookings:")
        for b in self.bookings:
            print(f"PNR: {b['PNR']}, {b['Passenger_Name']} ({b['Age']} {b['Gender']}) - {b['Train_Name']} ({b['Train_No']}) | {b['From']} ➝ {b['To']} | Class: {b['Class']}, ₹{b['Fare']} | Status: {b['Status']}")

    def search_booking_by_pnr(self):
        pnr = input("Enter PNR to search: ").strip()
        found = next((b for b in self.bookings if b['PNR'] == pnr), None)
        if found:
            print("\n🔎 Booking Details:")
            for key, value in found.items():
                print(f"{key}: {value}")
        else:
            print("No booking found with that PNR.")

    def run(self):
        while True:
            print("\n1. View All Trains\n2. Search Trains\n3. Find Train by Number\n4. Add New Train\n5. Delete Train")
            print("6. Book a Train\n7. Exit\n8. View All Bookings\n9. Search Booking by PNR")
            choice = input("Choose an option: ").strip()
            if choice == '1':
                self.display_trains()
            elif choice == '2':
                by = input("Search by (Starts/Ends/Name/Number): ").strip()
                query = input("Enter search term: ").strip()
                results = self.search_trains(query, by=by)
                self.display_trains(results)
            elif choice == '3':
                num = input("Enter train number: ").strip()
                train = self.find_train_by_number(num)
                if train is not None:
                    print(f"\n{train['Train name']} ({train['Train no.']}) | {train['Starts']} ➝ {train['Ends']}")
                else:
                    print("Train not found.")
            elif choice == '4':
                self.add_train(input("Train Number: "), input("Train Name: "), input("Start: "), input("End: "))
            elif choice == '5':
                self.delete_train(input("Enter train number to delete: ").strip())
            elif choice == '6':
                self.book_train()
            elif choice == '7':
                self.save_data()
                break
            elif choice == '8':
                self.view_all_bookings()
            elif choice == '9':
                self.search_booking_by_pnr()
            else:
                print("Invalid choice.")

if __name__ == "__main__":
    path = "/content/All_Indian_Trains.csv"
    if not os.path.exists(path):
        print(f"Error: {path} not found.")
        exit(1)
    system = RailwaySystem(path)
    system.run()


Loaded 4024 trains from /content/All_Indian_Trains.csv

1. View All Trains
2. Search Trains
3. Find Train by Number
4. Add New Train
5. Delete Train
6. Book a Train
7. Exit
8. View All Bookings
9. Search Booking by PNR
Choose an option: 6
Enter Train Number to book: 12033
Passenger Name: Apoorv
Passenger Age: 21
Passenger Gender (M/F/O): male
1. Sleeper (₹500)
2. 3rd AC (₹1200)
3. 2nd AC (₹1800)
4. 1st AC (₹2500)
Choose Class (1-4): 4

✅ Booking Confirmed! PNR: 4958286033

1. View All Trains
2. Search Trains
3. Find Train by Number
4. Add New Train
5. Delete Train
6. Book a Train
7. Exit
8. View All Bookings
9. Search Booking by PNR
Choose an option: 9
Enter PNR to search: 4958286033

🔎 Booking Details:
PNR: 4958286033
Booking_Date: 2025-06-11 06:34:31
Train_No: 12033
Train_Name: Cnb Ndls Shatabdi
From: Kanpur Central
To: New Delhi
Passenger_Name: Apoorv
Age: 21
Gender: MALE
Class: 1st AC
Fare: 2500
Status: Confirmed

1. View All Trains
2. Search Trains
3. Find Train by Number
4. Add N